<a href="https://colab.research.google.com/github/kiran9846/gpu_kmeans_clustering/blob/main/gpu_kmeans_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/kiran9846/gpu_kmeans_clustering.git

Cloning into 'gpu_kmeans_clustering'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), 34.72 KiB | 17.36 MiB/s, done.


In [ ]:
!cd gpu_kmeans_clustering/

In [ ]:
!ls

gpu_kmeans_clustering  sample_data


In [ ]:
!ls gpu_kmeans_clustering/

data-2.txt  README.md


In [ ]:
%cd gpu_kmeans_clustering/

/content/gpu_kmeans_clustering


In [ ]:
%ls

data-2.txt  README.md


In [ ]:
%%writefile vec.h
#ifndef VEC_H
#define VEC_H

#include <stdint.h>
#ifdef __cplusplus
extern "C" {
#endif

//__host__ __device__ void vec_zero(double v[], int dim);
__host__ int vec_read_stdin(double v[], int dim);
__host__ __device__ void vec_add(double u[], double v[], double w[], int dim);
__host__ __device__ void vec_scalar_mult(double v[], double c, double w[], int dim);
__host__ void vec_print(double v[], char* format, int dim);
__host__ __device__ double vec_norm_sq(double v[], int dim);

__host__ __device__ double vec_dist_sq(double *a, double *b, uint64_t dim);

__host__ __device__ void vec_copy(double *dest, double *src, uint64_t dim);

#ifdef __cplusplus
}
#endif

#endif


Writing vec.h


In [ ]:
%%writefile vec.cu
#include <stdio.h>
#include <stdint.h>
#include <cuda.h>
#include <float.h>
#include "vec.h"


// v = 0
__host__ __device__ void vec_zero (double v[], int dim) {
    for (int i=0; i<dim; i++) {
        v[i] = 0;
    }
}

// read a vector from stdin
// returns the number of elements read in
__host__ int vec_read_stdin (double v[], int dim) {
    for (int i=0; i<dim; i++) {
        if (scanf("%lf", &(v[i])) != 1) { // could also use v+i
            return i;
        }
    }
    return dim;
}

// w = u + v
__host__ __device__ void vec_add (double u[], double v[], double w[], int dim) {
    for (int i=0; i<dim; i++) {
        w[i] = u[i] + v[i];
    }
}

// w = cv
__host__ __device__ void vec_scalar_mult (double v[], double c, double w[], int dim) {
    for (int i=0; i<dim; i++) {
        w[i] = c * v[i];
    }
}

// print a vector using the given format specifier
__host__ void vec_print (double v[], char* format, int dim) {
    for (int i=0; i<dim; i++) {
        printf (format, v[i]);
    }
    printf ("\n");
}

// calculate the norm squared of a vector
__host__ __device__ double vec_norm_sq (double v[], int dim){
    double norm_sq = 0;
    for (int i=0; i<dim; i++) {
        norm_sq += v[i] * v[i];
    }
    return norm_sq;
}

// calculate the distance squared between two vectors
extern "C"{
__host__ __device__ double vec_dist_sq(double *a, double *b, uint64_t dim){
    double dist_sq = 0.0;
    for (uint64_t i = 0; i < dim; i++) {
        double diff = a[i] - b[i];
        dist_sq += diff * diff;
    }
    return dist_sq;
  }
}

// performs the copy v->data[i] = w->data[i] for all i
__host__ __device__ void vec_copy(double *dest, double *src, uint64_t dim) {
    for (uint64_t i = 0; i < dim; i++) {
        dest[i] = src[i];
    }
}



Writing vec.cu


In [ ]:
%%writefile gpu_kmeans_clust.h
#ifndef GPU_KMEANS_CLUST_H
#define GPU_KMEANS_CLUST_H

#include <stdint.h>

#ifdef __cplusplus
extern "C" {
#endif

__global__ void calc_arg_max(double *data, uint64_t num_points, uint64_t dim, uint64_t *centers, uint64_t m, uint64_t T, uint64_t* arg_max);
__global__ void find_cluster(double *kmeans, double *data, uint64_t num_points, uint64_t k, uint64_t dim, uint64_t *cluster);
__global__ void calc_new_centroids(double *data, uint64_t *cluster_assignments, double *kmeans_next, uint64_t num_points, uint64_t dim, uint64_t k, uint64_t *cluster_counts);
void calc_kmeans_next(double *data, uint64_t num_points, uint64_t dim, double *kmeans, double *kmeans_next, uint64_t k);
void calc_kmeans(double *data, uint64_t num_points, uint64_t dim, double *kmeans, uint64_t k, uint64_t m);

#ifdef __cplusplus
}
#endif

#endif


Writing gpu_kmeans_clust.h


In [ ]:
%%writefile gpu_kmeans_clust.cu
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <inttypes.h>
#include <cuda.h>
#include <float.h>
//#include "vec.h"
#include "vec.cu"

// calculate the arg max
__global__ void calc_arg_max (double *data, uint64_t num_points, uint64_t dim, uint64_t *centers, uint64_t m, uint64_t T, uint64_t* arg_max) {
    uint64_t thread_num = (uint64_t)blockIdx.x * blockDim.x + threadIdx.x;
    uint64_t thread_arg_max = 0;
    uint64_t start = thread_num * T;
    uint64_t end = start + T;
    if(end > num_points){
      end = num_points;
    }
    double cost_sq = 0;
    for (uint64_t i=start;i<end;i++) {
        double min_dist_sq = DBL_MAX;
        for (int j=0;j<m;j++) {
            double dist_sq = vec_dist_sq(data+i*dim,data+centers[j]*dim,dim);
            if (dist_sq < min_dist_sq) {
                min_dist_sq = dist_sq;
            }
        }
        if (min_dist_sq > cost_sq) {
            cost_sq = min_dist_sq;
            thread_arg_max = i;
        }
    }
    atomicMax((unsigned long long int *)arg_max,(unsigned long long int)thread_arg_max);
}

// find the index of the cluster for the given point
__global__ void find_cluster (double *kmeans, double *data, uint64_t num_points,uint64_t k, uint64_t dim, uint64_t *cluster) {
    uint64_t thread_num = (uint64_t)blockIdx.x * blockDim.x + threadIdx.x;
    if(thread_num < num_points){
      uint64_t thread_cluster = 0;
      double min_dist_sq = DBL_MAX;
      for(int i = 0; i < k; i++){
        double dist_sq = vec_dist_sq(data + thread_num * dim, kmeans + i * dim, dim);
        if(dist_sq < min_dist_sq){
            min_dist_sq = dist_sq;
            thread_cluster = i;
         }
       }
       cluster[thread_num] = thread_cluster;

    }
}
__global__ void calc_new_centroids(double *data, uint64_t *cluster_assignments, double *kmeans_next, uint64_t num_points, uint64_t dim, uint64_t k, uint64_t *cluster_counts) {
    extern __shared__ double shared_mem[];
    double *shared_sums = shared_mem;
    uint64_t *shared_counts = (uint64_t *)&shared_sums[k * dim];

    uint64_t idx = blockIdx.x * blockDim.x + threadIdx.x;
    uint64_t tid = threadIdx.x;

    // Initialize shared memory
    for (uint64_t i = tid; i < k * dim; i += blockDim.x) {
        shared_sums[i] = 0.0;
    }
    for (uint64_t i = tid; i < k; i += blockDim.x) {
        shared_counts[i] = 0;
    }
    __syncthreads();

    // Aggregate cluster sums and counts in shared memory
    if (idx < num_points) {
        uint64_t cluster = cluster_assignments[idx];
        for (uint64_t j = 0; j < dim; j++) {
            atomicAdd((unsigned long long int *)&shared_sums[cluster * dim + j], data[idx * dim + j]);
        }
        atomicAdd((unsigned long long int *)&shared_counts[cluster], 1);
    }
    __syncthreads();

    // Transfer shared memory to global memory
    for (uint64_t i = tid; i < k * dim; i += blockDim.x) {
        atomicAdd((unsigned long long int *)&kmeans_next[i], shared_sums[i]);
    }
    for (uint64_t i = tid; i < k; i += blockDim.x) {
        atomicAdd((unsigned long long int *)&cluster_counts[i], shared_counts[i]);
    }
}


// calculate the next kmeans
void calc_kmeans_next(double *data, uint64_t num_points, uint64_t dim, double *kmeans, double *kmeans_next, uint64_t k) {
    int blockSize = 256;  // Number of threads per block
    int numBlocks = (num_points + blockSize - 1) / blockSize;  // Number of blocks needed to cover all data points

    // Allocate memory for cluster assignments and counts on GPU
    uint64_t *d_cluster_assignments;
    double *d_data, *d_kmeans, *d_kmeans_next;
    uint64_t *d_cluster_counts;

    cudaMalloc(&d_cluster_assignments, num_points * sizeof(uint64_t));
    cudaMalloc(&d_data, num_points * dim * sizeof(double));
    cudaMalloc(&d_kmeans, k * dim * sizeof(double));
    cudaMalloc(&d_kmeans_next, k * dim * sizeof(double));
    cudaMalloc(&d_cluster_counts, k * sizeof(uint64_t));

    cudaMemcpy(d_data, data, num_points * dim * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_kmeans, kmeans, k * dim * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemset(d_kmeans_next, 0, k * dim * sizeof(double));
    cudaMemset(d_cluster_counts, 0, k * sizeof(uint64_t));

    // Launch kernels
    find_cluster<<<numBlocks, blockSize>>>(d_data, d_kmeans, num_points, dim, k, d_cluster_assignments);

    size_t shared_mem_size = k * dim * sizeof(double) + k * sizeof(uint64_t);
    calc_new_centroids<<<numBlocks, blockSize, shared_mem_size>>>(d_data, d_cluster_assignments, d_kmeans_next, num_points, dim, k, d_cluster_counts);

    // Copy results back to host
    cudaMemcpy(kmeans_next, d_kmeans_next, k * dim * sizeof(double), cudaMemcpyDeviceToHost);
    uint64_t *cluster_counts = (uint64_t *)malloc(k * sizeof(uint64_t));
    cudaMemcpy(cluster_counts, d_cluster_counts, k * sizeof(uint64_t), cudaMemcpyDeviceToHost);

    // Normalize the centroids
    for (uint64_t i = 0; i < k; i++) {
        if (cluster_counts[i] == 0) {
            printf("Error: Found an empty cluster. \n");
            exit(EXIT_FAILURE);
        }
        for (uint64_t j = 0; j < dim; j++) {
            kmeans_next[i * dim + j] /= cluster_counts[i];
        }
    }

    // Free GPU memory
    cudaFree(d_cluster_assignments);
    cudaFree(d_data);
    cudaFree(d_kmeans);
    cudaFree(d_kmeans_next);
    cudaFree(d_cluster_counts);
    free(cluster_counts);
}


// calculate kmeans using m steps of Lloyd's algorithm
void calc_kmeans(double *data, uint64_t num_points, uint64_t dim, double *kmeans, uint64_t k, uint64_t m) {
    int blockSize = 256;  // Number of threads per block
    int numBlocks = (num_points + blockSize - 1) / blockSize;  // Number of blocks needed to cover all data points

    // Find k centers using the farthest first algorithm
    uint64_t centers[k];
    centers[0] = 0;
    uint64_t arg_max;
    for (uint64_t m = 1; m < k; m++) {
        calc_arg_max<<<numBlocks, blockSize>>>(data, num_points, dim, centers, m, (num_points + blockSize - 1) / blockSize, &arg_max);
        cudaMemcpy(&centers[m], &arg_max, sizeof(uint64_t), cudaMemcpyDeviceToHost);
    }

    // Initialize kmeans using the k centers
    for (uint64_t i = 0; i < k; i++) {
        vec_copy(kmeans + i * dim, data + centers[i] * dim, dim);
    }

    // Update kmeans m times
    double *kmeans_next = (double *)malloc(k * dim * sizeof(double));
    for (uint64_t i = 0; i < m; i++) {
        calc_kmeans_next(data, num_points, dim, kmeans, kmeans_next, k);
        vec_copy(kmeans, kmeans_next, k * dim);
    }
    free(kmeans_next);
}






Writing gpu_kmeans_clust.cu


In [ ]:
%%writefile main.cu
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <inttypes.h>
#include <cuda.h>
#include <float.h>
#include "gpu_kmeans_clust.h"
#include "vec.h"


int main(int argc, char* argv[]) {
    // get k and m from command line
    if (argc < 3) {
        printf("Command usage : %s %s %s\n", argv[0], "k", "m");
        return 1;
    }
    uint64_t k = atoi(argv[1]);
    uint64_t m = atoi(argv[2]);

    // read the number of points and the dimension of each point
    uint64_t num_points, dim;
    if (scanf("%" PRIu64 " %" PRIu64, &num_points, &dim) != 2) {
        printf("error reading the number of points and the dimension\n");
        return 1;
    }

    // dynamically allocate memory for the data array
    double* data = (double*)malloc(num_points * dim * sizeof(double));

    // Read vectors from stdin and store them in the data array
    for (uint64_t i = 0; i < num_points; i++) {
        if (vec_read_stdin(data + i * dim, dim) != dim) {
            printf("error reading the next point from stdin\n");
            return 1;
        }
    }

    // calculate kmeans using m steps of Lloyd's algorithm
    double *kmeans = (double *)malloc(k * dim * sizeof(double));
    calc_kmeans(data, num_points, dim, kmeans, k, m);

    // print the results
    for (uint64_t i = 0; i < k; i++) {
        for (uint64_t j = 0; j < dim; j++) {
            printf("%.5f ", kmeans[i * dim + j]);
        }
        printf("\n");
    }

    // free the dynamically allocated memory
    free(data);
    free(kmeans);

    return 0;
}

Writing main.cu


In [ ]:
!nvcc -c vec.cu -o vec.o

In [ ]:
!nvcc -c gpu_kmeans_clust.cu -o gpu_kmeans_clust.o

In [ ]:
!nvcc -c main.cu -o main.o

In [ ]:
!nvcc main.o gpu_kmeans_clust.o vec.o -o my_cuda_program

/usr/bin/ld: vec.o: in function `vec_zero(double*, int)':
tmpxft_00000481_00000000-6_vec.cudafe1.cpp:(.text+0x1a): multiple definition of `vec_zero(double*, int)'; gpu_kmeans_clust.o:tmpxft_000004a6_00000000-6_gpu_kmeans_clust.cudafe1.cpp:(.text+0x1a): first defined here
/usr/bin/ld: vec.o: in function `vec_read_stdin':
tmpxft_00000481_00000000-6_vec.cudafe1.cpp:(.text+0x5e): multiple definition of `vec_read_stdin'; gpu_kmeans_clust.o:tmpxft_000004a6_00000000-6_gpu_kmeans_clust.cudafe1.cpp:(.text+0x5e): first defined here
/usr/bin/ld: vec.o: in function `vec_add':
tmpxft_00000481_00000000-6_vec.cudafe1.cpp:(.text+0xc5): multiple definition of `vec_add'; gpu_kmeans_clust.o:tmpxft_000004a6_00000000-6_gpu_kmeans_clust.cudafe1.cpp:(.text+0xc5): first defined here
/usr/bin/ld: vec.o: in function `vec_scalar_mult':
tmpxft_00000481_00000000-6_vec.cudafe1.cpp:(.text+0x141): multiple definition of `vec_scalar_mult'; gpu_kmeans_clust.o:tmpxft_000004a6_00000000-6_gpu_kmeans_clust.cudafe1.cpp:(.te

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!git pull

Already up to date.


In [ ]:
!time ./gpu_kmeans 3 10

/bin/bash: line 1: ./gpu_kmeans: No such file or directory

real	0m0.001s
user	0m0.000s
sys	0m0.001s
